# Project 3

Francesca Michielli (fcm2118)
- **Datasets to be used:**
  - [ACLED Database](https://acleddata.com/data-export-tool/) Date range: 1/1/2018 - 11/1/2024
  - [Mexico Population Density by Region](https://www.statista.com/statistics/1323129/mexico-population-density-by-state/)   
  
  The two databases I will be using are the ACLED database and Mexico population density by region. 
  ACLED is an organization that collects ["data on violent conflict and protest"](https://acleddata.com/) 
  events. For each event in its database, it records the location, including the country, 
  region, and goegraphic coordinates; disorder type, event type, and sub-type of event 
  (ex: Demonstrations, Protest, and Peaceful protests); number of fatalities; 
  the actors involved; and notes.
- **Analysis question:**    

  How does the number, distribution, and intensity of conflict and protest events differ in different regions of Mexico? How is this related to population density? Are conflict and protest events more common in more densely populated areas?

- **Columns that will (likely) be used:**
  - Event country and location ('country', 'admin1', and 'State')
  - Event date (event_date)
  - Disorder type and event type ('disorder_type'	'event_type')
  - Population Density ('Population density in Mexico in 2020, by federal entity (inhabitants per square kilometer)')

- **Columns to be used to merge/join datasets:**
  - ACLED Data: admin1 (this shows the state)
  - Population Data: State

- **Hypothesis**: More conflict events will take place in more densely populated states. The locations with more events will have more associated fatalities. 

- **Site URL:** [URL from Publish section]

In [1]:
import plotly.io as pio

pio.renderers.default = "vscode+jupyterlab+notebook_connected"

gH3HesSGHwUhV5U-pob!

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### 1. Read in and clean ACLED data
- ACLED's ["event-based dataset is designed for disaggregated conflict analysis and crisis mapping"](https://acleddata.com/data/)
- Each observation represents a conflict or protest event

__GitHub Note__   
The ACLED dataset that I originally downloaded is very large, almost 2 million observations. Because this file is too large to upload to GitHub, I cleaned it in a separate file. Below are the commands I used to filter the data down to only observations in Mexico in 2024.   
`ACLED = pd.read_csv("ACLED.csv")`   
`ACLED.info()`   
- This displays the number of observations (1959833) and the datatype of the event date (object)
  
`ACLED['event_date'] = pd.to_datetime(ACLED['event_date'], format = '%d %B %Y')`   
- This changes the data type of event date to date-time

`mexicoACLED = ACLED[(ACLED['event_date'] >= "2024-01-01") & (ACLED['country'] == "Mexico")]`  
- This filters the data to observations that took place in 2024 and in Mexico

`mexicoACLED.to_csv('mexicoACLED.csv', index=False)`
- This saves the dataframe as a csv

In [3]:
mexicoACLED = pd.read_csv('mexicoACLED.csv')

In [4]:
mexicoACLED.sample(5)

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp,population_best
5198,MEX89516,2024-07-03,2024,1,Political violence,Violence against civilians,Attack,Unidentified Armed Group (Mexico),NaN,Political militia,...,20.7206,-103.3883,1,Pagina 24 Jalisco,Subnational,"On 3 July 2024, in Zapopan, Jalisco, according...",1,NaN,1720481996,286063.0
4438,MEX90550,2024-07-22,2024,1,Political violence,Violence against civilians,Attack,Unidentified Gang (Mexico),NaN,Political militia,...,18.9218,-99.2349,1,El Universal (Mexico),National,"On 22 July 2024, in Cuernavaca, Morelos, two m...",0,NaN,1724714228,28625.0
7620,MEX87240,2024-05-09,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Mexico),Labor Group (Mexico),Protesters,...,25.0838,-108.0580,1,Noroeste,Subnational,"On 9 May 2024, in La Reforma, Sinaloa, employe...",0,crowd size=no report,1715641795,2915.0
11129,MEX83699,2024-02-05,2024,3,Political violence,Violence against civilians,Attack,Unidentified Gang (Mexico),NaN,Political militia,...,19.0613,-98.3061,1,E-Consulta,Subnational,"Around 5 February 2024 (month of), in Cholula ...",1,NaN,1719274856,27608.0
10905,MEX84065,2024-02-10,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Mexico),NaN,Protesters,...,25.6647,-100.3110,1,El Norte,Subnational,"On 10 February 2024, in Monterrey, Nuevo Leon,...",0,crowd size=around 50 people,1708383904,13532.0


#### 1.A. Preliminary Map of ACLED Events

__Create a scatter map of ACLED events in 2024__
- The map below shows each ALCED event in 2024 in Mexico.
- The colors of the dots differentiates the event type

In [5]:
import plotly.express as px
fig = px.scatter_geo(
        mexicoACLED,
        lat ="latitude",
        lon = "longitude",
        color = 'event_type',
        center = {'lat': 23.6345, 'lon' :-102.5528},
        title = "ACLED Events in Mexico by Event Type, 2024"
)
fig.update_geos(
    projection_scale=8
    )

__Conclusions from the Map:__
1. The clustering of the dots make it hard to draw many conclusions, yet we can still see some patterns.
1. It appears there was a lot of violence in the center of the country, __near Mexico City__. 
In particular, there were many riots, protests, and incidents of violence against civilains. 
1. Along the western coast there is a clustering of strategic developments.
1. Near the border with the US there are relatively few events

### 2. Read in population data

In [6]:
pop_density = pd.read_excel("Mexico Pop Density.xlsx")

Preview the data:

In [7]:
pop_density.head()

,State,"Population density in Mexico in 2020, by federal entity (inhabitants per square kilometer)"
0,Mexico City,6163.3
1,State of Mexico,760.2
2,Morelos,404.1
3,Tlaxcala,336.0
4,Aguascalientes,253.9


Rename columns:

In [8]:
pop_density.columns


Index(['State ', 'Population density in Mexico in 2020, by federal entity (inhabitants per square kilometer)'], dtype='object')

In [9]:

pop_density.rename(columns={'State ': 'State',
                            'Population density in Mexico in 2020, by federal entity (inhabitants per square kilometer)': 'Pop Density'},
                              inplace=True)

### 3. Reshape the Data

The data must be regrouped in order to compare the number of events per region with the population density.   
The following code __groups the data by state and event type__ and shows the __count of each event per state__.

In [10]:
mexico_group = mexicoACLED.groupby(["admin1", "event_type"]).count().reset_index()

mexico_group.head()

,admin1,event_type,event_id_cnty,event_date,year,time_precision,disorder_type,sub_event_type,actor1,assoc_actor_1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp,population_best
0,Aguascalientes,Battles,3,3,3,3,3,3,3,0,...,3,3,3,3,3,3,3,0,3,3
1,Aguascalientes,Protests,11,11,11,11,11,11,11,9,...,11,11,11,11,11,11,11,11,11,11
2,Aguascalientes,Strategic developments,6,6,6,6,6,6,6,0,...,6,6,6,6,6,6,6,0,6,0
3,Aguascalientes,Violence against civilians,11,11,11,11,11,11,11,0,...,11,11,11,11,11,11,11,0,11,11
4,Baja California,Battles,48,48,48,48,48,48,48,0,...,48,48,48,48,48,48,48,1,48,48


### 4. Merge the Data
- Merge the two datasets based on the state/region
- Specifically, merge the population density dataset in with the ACLED dataset
- Keep rows that are not found in both datasets by specifying "outer"

In [11]:
mexico_merge = pd.merge(mexico_group[["admin1", "event_type", "event_id_cnty"]],
                        pop_density,
                        left_on = "admin1",
                        right_on = "State",
                        how = 'outer' )

Preview the data:

In [12]:
mexico_merge.head()

,admin1,event_type,event_id_cnty,State,Pop Density
0,Aguascalientes,Battles,3.0,Aguascalientes,253.9
1,Aguascalientes,Protests,11.0,Aguascalientes,253.9
2,Aguascalientes,Strategic developments,6.0,Aguascalientes,253.9
3,Aguascalientes,Violence against civilians,11.0,Aguascalientes,253.9
4,Baja California,Battles,48.0,Baja California,52.8


Look for unique population density values:

In [13]:
mexico_merge["Pop Density"].unique()

array([ 253.9,   52.8,   10.8,   16.1,   75.6,   15.1,    nan,   20.8,
        130. ,   14.9,  201.5,   55.7,  148.1,  106.2, 6163.3,   81. ,
        404.1,   44.4,   90.2,   44.1,  191.9,  202.6,   41.6,   46.2,
         16.4,  760.2,   97.1,   44. ,  336. ,  112.3,   58.7,   21.5])

From the code above, it is clear that the data did not merge properly. 
- This is because there is are 'nan' values for population density. 
- The code below displays the __state names for the ACLED events that were not assigned a population density.__

In [14]:
mexico_merge[mexico_merge["Pop Density"].isnull()]["admin1"].unique()

array(['Ciudad de Mexico', 'Mexico', 'Michoacan de Ocampo', 'Nuevo Leon',
       'Queretaro', 'San Luis Potosi', 'Veracruz de Ignacio de la Llave',
       'Yucatan'], dtype=object)

Compare this to the unique names for states in the population density dataset:

In [15]:
pop_density['State'].unique()

array(['Mexico City', 'State of Mexico', 'Morelos', 'Tlaxcala',
       'Aguascalientes', 'Querétaro', 'Guanajuato', 'Puebla', 'Hidalgo',
       'Colima', 'Veracruz', 'Jalisco', 'Tabasco', 'Nuevo León',
       'Michoacán', 'Chiapas', 'Yucatán', 'Guerrero', 'Sinaloa',
       'Baja California', 'San Luis Potosí', 'Nayarit', 'Oaxaca',
       'Tamaulipas', 'Quintana Roo', 'Zacatecas', 'Coahuila de Zaragoza',
       'Sonora', 'Campeche', 'Chihuahua', 'Durango',
       'Baja California Sur'], dtype=object)

From a visual comparison, it is clear that the population density data did not merge correctly because the __names for a subset of the states does not match across the datasets__. To remedy this, replace the names so they match:

In [16]:
mexico_group = mexico_group.replace({"Ciudad de Mexico": "Mexico City",
        "Mexico": "State of Mexico",
         "Michoacan de Ocampo" : "Michoacán",
         "Nuevo Leon" : "Nuevo León",
         "Queretaro" : "Querétaro",
         "San Luis Potosi" : "San Luis Potosí",
         "Veracruz de Ignacio de la Llave" : "Veracruz",
         "Yucatan" : "Yucatán"
         })


In [17]:
mexicoACLED = mexicoACLED.replace({"Ciudad de Mexico": "Mexico City",
        "Mexico": "State of Mexico",
         "Michoacan de Ocampo" : "Michoacán",
         "Nuevo Leon" : "Nuevo León",
         "Queretaro" : "Querétaro",
         "San Luis Potosi" : "San Luis Potosí",
         "Veracruz de Ignacio de la Llave" : "Veracruz",
         "Yucatan" : "Yucatán"
         })

Merge the corrected datasets:

In [18]:
mexico_merge = pd.merge(mexico_group, pop_density, left_on = "admin1", right_on = "State", how = 'outer' )

There are no more 'nan' values, so the data merged successfully:

In [19]:
mexico_merge["Pop Density"].unique()

array([ 253.9,   52.8,   10.8,   16.1,   75.6,   15.1,   20.8,  130. ,
         14.9,  201.5,   55.7,  148.1,  106.2, 6163.3,   81. ,  404.1,
         44.4,   90.2,   44.1,  191.9,  202.6,   41.6,   46.2,   16.4,
        760.2,   97.1,   44. ,  336. ,  112.3,   58.7,   21.5])

Check to make sure population density is stored as a float:

In [20]:
mexico_merge["Pop Density"].dtype

dtype('float64')

Clean column names and drop redundant columns:

In [21]:
mexico_clean = mexico_merge[["admin1",
                            "event_type",
                            "event_id_cnty",
                            "Pop Density"]].rename(columns =
                                                    {"admin1" : "State" ,
                                                      "event_id_cnty": "count"})

In [22]:
mexico_clean.head()

,State,event_type,count,Pop Density
0,Aguascalientes,Battles,3,253.9
1,Aguascalientes,Protests,11,253.9
2,Aguascalientes,Strategic developments,6,253.9
3,Aguascalientes,Violence against civilians,11,253.9
4,Baja California,Battles,48,52.8


### 5. Analysis of Number of Events per State

First, create a chart that shows the ACLED events in Mexico in 2024 by region and event type.

In [23]:
bar = px.bar(
    mexico_clean,
    x = "State",
    y = "count",
    color = "event_type",
    title = "ACLED Events in Mexico 2024 by Region and Event Type",
    labels = {"event_type" : "Event Type"}
)

bar.show()

From the chart above it is apparent that a few states have higher number of civil unrest incidents.
__Specifically, Guanajuato, Guerrero, and Veracruz have the most events.__
- In Veracruz, Protests make up the majority of these events
- By contrast, in Guanajuato the majority of events were violence against civilians

Next, let's add on population density as a line on top of this bar chart:

In [24]:
bar.add_scatter(x=mexico_clean['State'], 
                y=mexico_clean['Pop Density'], 
                mode='lines', 
                name='Population Density')

This is somewhat helpful, but __Mexico City is a big outlier__ in population density.
- It will be more helpful to view the other states without Mexico City

First, lets make a function to generate these charts. The function below: 
- Charts ACLED events by event type as bars
- Charts population density as a line
- Specifies the color of each event type so there is consistency across charts

In [25]:
def chart_data(df,title=""):
    bar2 = px.bar(
        df,
        x = "State",
        y = "count",
        color = "event_type",
        color_discrete_map={
                "Battles": "red",
                "Protests": "turquoise",
                "Strategic developments": "blue",
                "Violence against civilians": "yellow",
                "Riots": "magenta",
                "Explosions/Remote violence" : "orange"},
        title = f"ACLED Events in Mexico 2024 by Region and Event Type with Population Density {title}",
        labels = {"event_type" : "Event Type"}
    )

    bar2.add_scatter(x=df['State'], 
                    y=df['Pop Density'], 
                    mode='lines', 
                    name='Population Density', 
                    line_color='black'
                    )

    bar2.show()

Now we can generate the chart with the same data, but excluding Mexico City:

In [26]:
mexico_noMC = mexico_clean[mexico_clean["State"] != "Mexico City"]

In [27]:
chart_data(mexico_noMC, "without Mexico City")

This is much easier to see: 
- We can see that Guerrero, which has the most ACLED events has a relatively low population density.
- By contrast, the State of Mexico, which seems to have an average number of events, has the highest population density in this sample.

__The following two charts__ separate out protest events from non-protest events:

In [28]:
chart_data(mexico_noMC[mexico_noMC["event_type"] == "Protests"],"- Protests Only")

In [29]:
chart_data(mexico_noMC[mexico_noMC["event_type"] != "Protests"], "- No Protests")

__Conclusions from charts:__

* From these charts, it is evident that __population density is not the only predictor of the number of conflict events in a state__. Some states have proportionately more conflict events, while other states have proportionately fewer.
* For example, Guerrero has a much smaller population density than Colima, yet has many more conflict events. Similarly, Sinaloa and Baja California have similar population densities, yet Sinaloa experienced many more conflict events. 
* On the other hand, the State of Mexico and Mexico City have very large population densities, with an average number of conflict events.
* This indicates that __some areas are more prone to violence and conflict than others,__ and that __the more contentious areas are not necessarily the biggest cities with the highest populations.__ 

### 6. Analysis of Fatalities from Conflict Events

To further this analysis and investigate the __severity__ of each event, we can look at the number of fatalities from conflict events. 

First, make sure that 'fatalities' is stored as an integer:

In [30]:
mexicoACLED["fatalities"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 12191 entries, 0 to 12190
Series name: fatalities
Non-Null Count  Dtype
--------------  -----
12191 non-null  int64
dtypes: int64(1)
memory usage: 95.4 KB


Next, group the data based on state and event type like before, but this time take the __sum__ of fatalities. Note that this also take the average value of latitude and longitude, which will be used later to approximate the latitude and longitude of each region. 

In [31]:
mexico_fatal = mexicoACLED.groupby(["admin1", "event_type"]).agg(
    { "fatalities" : "sum",
      'latitude':"mean",
      'longitude':"mean"}).reset_index()

mexico_fatal.head()

,admin1,event_type,fatalities,latitude,longitude
0,Aguascalientes,Battles,2,21.890500,-102.231867
1,Aguascalientes,Protests,0,21.879800,-102.296000
2,Aguascalientes,Strategic developments,0,22.102233,-102.300400
3,Aguascalientes,Violence against civilians,7,21.975855,-102.272727
4,Baja California,Battles,42,32.484156,-116.635460


Merge the fatalities column to the clean dataset: 

In [32]:
mexico_clean = pd.merge(mexico_clean,
                        mexico_fatal[['admin1', 'event_type','fatalities', 'latitude', 'longitude']],
                        left_on = ["State",	"event_type"],
                        right_on = ["admin1", "event_type"])

mexico_clean.head()

,State,event_type,count,Pop Density,admin1,fatalities,latitude,longitude
0,Aguascalientes,Battles,3,253.9,Aguascalientes,2,21.890500,-102.231867
1,Aguascalientes,Protests,11,253.9,Aguascalientes,0,21.879800,-102.296000
2,Aguascalientes,Strategic developments,6,253.9,Aguascalientes,0,22.102233,-102.300400
3,Aguascalientes,Violence against civilians,11,253.9,Aguascalientes,7,21.975855,-102.272727
4,Baja California,Battles,48,52.8,Baja California,42,32.484156,-116.635460


Drop unnecessary columns:

In [33]:
mexico_clean.drop('admin1', axis = 1)

,State,event_type,count,Pop Density,fatalities,latitude,longitude
0,Aguascalientes,Battles,3,253.9,2,21.890500,-102.231867
1,Aguascalientes,Protests,11,253.9,0,21.879800,-102.296000
2,Aguascalientes,Strategic developments,6,253.9,0,22.102233,-102.300400
3,Aguascalientes,Violence against civilians,11,253.9,7,21.975855,-102.272727
4,Baja California,Battles,48,52.8,42,32.484156,-116.635460
...,...,...,...,...,...,...,...
162,Zacatecas,Battles,42,21.5,36,22.760010,-102.609126
163,Zacatecas,Explosions/Remote violence,2,21.5,0,22.712750,-102.780850
164,Zacatecas,Protests,74,21.5,0,22.787723,-102.598124
165,Zacatecas,Strategic developments,29,21.5,0,22.872245,-102.579848


__Chart the fatalities per incident by incident type:__

In [34]:
def chart_data_fatal(df,title=""):
    bar2 = px.bar(
        df,
        x = "State",
        y = "fatalities",
        color = "event_type",
        color_discrete_map={
                "Battles": "red",
                "Protests": "turquoise",
                "Strategic developments": "blue",
                "Violence against civilians": "yellow",
                "Riots": "magenta",
                "Explosions/Remote violence" : "orange"},
        title = f"ACLED Fatalities in Mexico 2024 by Region and Event Type with Population Density {title}",
        labels = {"event_type" : "Event Type"}
    )

    bar2.add_scatter(x=df['State'], 
                    y=df['Pop Density'], 
                    mode='lines', 
                    name='Population Density', 
                    line_color='black'
                    )

    bar2.show()

Filter out Mexico City because it is an outlier:

In [35]:
mexico_noMC = mexico_clean[mexico_clean["State"] != "Mexico City"]

In [36]:
chart_data_fatal(mexico_noMC)

The chart above depicts the number of fatalities from conflict incidents by state and incident type
, as well as population density. __The majority of fatalities come from violence against
civilians;__ this is unsurprising, as these events are violent in nature and make up a large
proportion of the events that took place. 

Let's look at aggregate fatalities vs incidents:
- The following code groups the incidents by state, and provides a count of the 
number of incidents, a sum total of the number of fatalities, and the population density of the state.

In [51]:
mexico_ag = mexico_clean.groupby("State").agg(
    {"count" : "sum", "fatalities" : "sum",
      "Pop Density":"mean",
        'latitude':"mean",
        'longitude':"mean"}).reset_index()

mexico_ag

,State,count,fatalities,Pop Density,latitude,longitude
0,Aguascalientes,31,9,253.9,21.962097,-102.275248
1,Baja California,578,362,52.8,32.459206,-116.588650
2,Baja California Sur,83,4,10.8,24.641039,-110.743578
3,Campeche,153,20,16.1,19.240874,-90.851952
4,Chiapas,548,339,75.6,16.257999,-92.523068
5,Chihuahua,307,232,15.1,29.037541,-106.274494
6,Coahuila de Zaragoza,120,5,20.8,26.707115,-101.566070
7,Colima,226,182,130.0,19.165102,-103.839562
8,Durango,103,14,14.9,24.814584,-104.997855
9,Guanajuato,747,770,201.5,20.711187,-101.145528


Next, graph all of the data on one chart:
- The following chart shows the number of fatalities and number of conflict events by state as bars
- The population density is shown as a line
- Note that this also excludes Mexico City, as it is an outlier in population density

In [52]:
fig3 = px.bar(mexico_ag[mexico_ag['State'] != "Mexico City"], x = "State", 
       y = ["count", "fatalities"],
       barmode="group")

fig3.add_scatter(x=mexico_ag[mexico_ag['State'] != "Mexico City"]["State"], 
                    y=mexico_ag[mexico_ag['State'] != "Mexico City"]['Pop Density'], 
                    mode='lines', 
                    name='Population Density', 
                    line_color='black'
                    )

This is an interesting chart, but it will be easier to interpret when the observations are in descending order. The following code sorts the observations based on event count and replicates this chart.

In [53]:
mexico_ag_sorted = mexico_ag.sort_values("count", ascending= False)

In [54]:
mexico_ag_sorted

,State,count,fatalities,Pop Density,latitude,longitude
10,Guerrero,984,575,55.7,17.384510,-99.835598
29,Veracruz,884,259,112.3,19.094665,-96.513651
17,Nuevo León,763,676,90.2,25.648894,-100.223004
9,Guanajuato,747,770,201.5,20.711187,-101.145528
13,Mexico City,723,161,6163.3,19.396862,-99.141808
14,Michoacán,718,391,81.0,19.445598,-101.909077
23,Sinaloa,705,443,52.8,24.373776,-107.194245
19,Puebla,603,284,191.9,18.999706,-97.943644
1,Baja California,578,362,52.8,32.459206,-116.588650
4,Chiapas,548,339,75.6,16.257999,-92.523068


In [55]:
fig4 = px.bar(mexico_ag_sorted[mexico_ag_sorted['State'] != "Mexico City"], x = "State", 
       y = ["count", "fatalities"],
       barmode="group")

fig4.add_scatter(x=mexico_ag_sorted[mexico_ag_sorted['State'] != "Mexico City"]["State"], 
                    y=mexico_ag_sorted[mexico_ag_sorted['State'] != "Mexico City"]['Pop Density'], 
                    mode='lines', 
                    name='Population Density', 
                    line_color='black'
                    )

__This chart presents another way of assesing the risk of violence in__
__different Mexican states:__
* The State of Mexico, Tlaxcala, and Mexico City (not shown here) are two 
of the safest states in Mexico. This is evident in comparing the number of 
fatalities from conflict events to the population density. These three states
 have some of the highest population densities, and relatively fewer fatalities. 
* By comparison, Guanajuato, Tabasco, and Nuevo León have a relatively high
 number of fatalities, compared with the population density. 
* This data also highlights that the conflict events are the most severe in
 Guanajuato as this state has more fatalities than the number of conflict events. 
 By contrast, Veracruz, which has the second higest count of conflict events, has 
 many fewer fatalities, indicating relatively peaceful events. 

### 7. Map of ACLED Events by Intensity
The final section of this analysis provides a graphical
depiction of the conflict data. 

The following map shows __total conflict events__ in each state.
* The __color__ of the dots indicate the number of events
* The __size__ of the dots indicate the population density 

In [80]:
map2 = px.scatter_geo(
        mexico_ag_sorted,
        lat ="latitude",
        lon = "longitude",
        size = "Pop Density",
        color = "count",
        center = {'lat': 23.6345, 'lon' :-102.5528},
        title = "Number of ACLED Events in Mexico and Population Density, 2024",
        hover_name = "State",
        size_max = 50,
        color_continuous_scale="thermal_r"

)
map2.update_geos(
    projection_scale=8
    )

The following map shows __total fatalities from conflict events__ in each state.
* The __color__ of the dots indicate the number of events
* The __size__ of the dots indicate the population density 

In [85]:
map3 = px.scatter_geo(
        mexico_ag_sorted,
        lat ="latitude",
        lon = "longitude",
        size = "Pop Density",
        color = "fatalities",
        center = {'lat': 23.6345, 'lon' :-102.5528},
        title = "Fatalities from ACLED Events in Mexico and Population Density, 2024",
        hover_name = "State",
        size_max = 50,
        color_continuous_scale = "thermal_r"

)
map3.update_geos(
    projection_scale=8
    )

__These two maps complement the prior charts:__
* These maps show spatially where the most intense conflicts are located.
* At the same time, they show which states have the highest population density.
* This enables us to see where the most intense conflicts, as determined
by a comparison between the number of event/fatalities and population
density, are located: the smaller dots with darkest color are the most conflict
prone, whereas the bigger and lighter dots are relatively less conflict prone.
* Similar to the prior charts, these maps show us that Guerrero, Nuevo Leon,
 and Guanajuato have the most dangerous conflict events.
* This also indicates that Mexico City, which has a high number of conflict events, 
has a relatively low number of fatalities: its circle is large but color is light
in the secon map.

### 8. Conclusions
1. This analysis shows that population density and the number of conflict events are not perfectly correlated: some less populated regions have more and more-severe conflict events than regions that are more populated.